# KNIT7
# Demonstrate the use of Precision Timing Capability
In this experiment, we will start a tcpdump process to capture all ICMP request packets with a nano-second high precison timestamp obtained from the NIC/PHC Clock into `PCAP` files. We will then send 5 ping packets from Node1 to Node4 and once the ping operation is complete, we will gather the pcap files and perform an analysis 
on the packets timestamps by tracking each packet as it flows though all the hops to reach its destination.

## Install python-scapy library

In [ ]:
%%bash
pip install scapy

## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import requests
from scapy.all import *

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Get Slice Name

In [ ]:
slice_name=f"Slice for KNIT7 Precision Timing Tutorial"
slice = fablib.get_slice(name = slice_name)
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()

## Experiment Setup
1. Upload  bash scripts in tools directory that will perform tcpdumps
2. Start tcpdump on all nodes
3. Start a series of 5 pings from Node1 to Node4
4. Stop the tcpdump operation.
5. Gather all pcap files from the tcpdump operation

In [ ]:
nodes = slice.get_nodes()
node1 = None
for node in nodes:
    node.upload_directory('tools','~/')
    iface_name = None
    node_name = node.get_name()
    if(node_name in ['node1']):
        iface_name = (node.get_interfaces()[0]).get_physical_os_interface_name()
        node1 = node
    elif(node_name in ['node4']):
        iface_name = (node.get_interfaces()[0]).get_physical_os_interface_name()
    elif(node_name in ['node2']):
        iface_name = (node.get_interface(network_name='net1')).get_physical_os_interface_name()
    else:
        iface_name = (node.get_interface(network_name='net2')).get_physical_os_interface_name()
    print (f"Starting tcpdump on {node_name}")
    node.execute_thread(f"sudo -b tools/start_dump.sh {node_name} {iface_name}")
    
stdout,stderr = node1.execute(f"ping -c10 node4")

for node in nodes:
    node_name = node.get_name()
    print (f"Stop TCPDUMP and Download pcap file to {node_name}.pcap")
    stdout,stderr = node.execute(f"sudo -b tools/stop_dump.sh")
    #stdout,stderr = node.execute(f"sudo killall tcpdump")
    node.download_file(f'{node_name}.pcap',f'{node_name}.pcap')

## Analyze packet captures
As the ping packets traverse thru node2 and node3 to reach node4, we collect pcap files at each hop on the incomming interfaces. Using the packet traces we identify each packet and compare the packet capture timestamps. If the clocks on all NICS were synchrinized via GPS clocks, then we should see timestamps incrementing. If for some reason the clock on any one of the NICs is not synchronized, the timestamps may not sequentially match up and lead to miscalcuations in computing transit times at intermediate(or end) hops.

In [ ]:
node4_packets = rdpcap('node4.pcap')
node3_packets = rdpcap('node3.pcap')
node2_packets = rdpcap('node2.pcap')
node1_packets = rdpcap('node1.pcap')

def compare_packet(my_packet,other_packets):
    for packet in other_packets:
        if (my_packet[IP].src == packet[IP].src) and (my_packet[IP].dst == packet[IP].dst) and (my_packet[IP].id == packet[IP].id):
            return packet
    return None
        
    
for node1_packet in node1_packets:
    node2_packet = compare_packet(node1_packet,node2_packets)
    node3_packet = compare_packet(node1_packet,node3_packets)
    node4_packet = compare_packet(node1_packet,node4_packets)
    if node2_packet is not None and node3_packet is not None and node4_packet is not None:
        print (f"{node1_packet.summary()} with ICMP SEQ# {node1_packet[ICMP].seq} found exiting \n \
        node1 at {node1_packet.time} and then entering \n \
        node2 at {node2_packet.time} and then entering \n \
        node3 at {node3_packet.time} and then entering \n \
        node4 at {node4_packet.time} and then entering \n \
        \n\n")